![intro_banner](./Images/27-5-2.3-Banner.png)

---

# Introduction

In this exercise you will practice python programming applied to Earth Observation, within the VRE

This exercise is inspired by a research article by M. Urban et. al. that you can download from [this link](https://www.mdpi.com/2072-4292/10/9/1482)

During the southern summer season of 2015 and 2016, South Africa experienced one of the most severe meteorological droughts since the start of climate recording, due to an exceptionally strong El Niño event. To investigate spatiotemporal dynamics of surface moisture and vegetation structure, data from ESA’s Copernicus Sentinel-1/-2 and NASA’s Landsat-8 for the period between March 2015 and November 2017 were utilized

Several techniques and indices were computed to assess the severity of the drought and their temporal variations.

Here, we will limit the study to the computation of the vegetation index to compare vegetation status between dry and wet seasons, both visually and numerically.

The NDWI will also be used to visualise variation in water level.

# Import all the modules needed for the exercise

In [ ]:
import os
import warnings

import xarray
import rioxarray
import rasterio
from rasterio.plot import show
import numpy as np
import folium

from eodag import EODataAccessGateway

warnings.filterwarnings("ignore")

# Fetch the products with EODAG

Execute the cell below to get the products

In [ ]:
dag = EODataAccessGateway()
product_type = "S2_MSI_L1C"
footprint = {"lonmin": 31, "latmin": -26, "lonmax": 32, "latmax": -25}
cloud_cover = 4
start, end = "2016-08-31", "2017-02-01"
search_results, estimated_total_nbr_of_results = dag.search(
    productType=product_type,
    geom=footprint,
    start=start,
    end=end,
    cloudCover=cloud_cover,
)

To make sure we work with the same products, the name of the two products to download is given below

In [ ]:
# Search results by name
title1 = "S2A_MSIL1C_20170119T074231_N0204_R092_T36JUT_20170119T075734"
title2 = "S2A_MSIL1C_20160901T073612_N0204_R092_T36JUT_20160901T080536"

Find the indices of the products corresponding to the titles above

Hint : use Python's built-in function **enumerate**

In [ ]:
for i, p in enumerate(search_results):
    if p.properties["title"] == title1 or p.properties["title"] == title2:
        print((p.properties["cloudCover"], p.properties["id"], i))

EODAG search results properties are stored in the form of a dictionnary that you can access via **.properties** 

These properties inlude the name of the instrument, the platform, dates, title, description of the bands, etc.

Hint : search_result is a list of EODAG products

In [ ]:
# Print list of properties
search_results[0].properties

_**Exercise :**_

Display list of products having a cloud cover lower than 20%

Remember that you can access the list of properties with the .properties method

In [ ]:
for i, p in enumerate(search_results):
    if p.properties["cloudCover"] < 20:
        print(
            "Product index: {0} - Cloud cover {1}%, ".format(
                i, p.properties["cloudCover"]
            )
        )

Now we know exactly which product to download, we can launch the download process

If you did the search properly you have found that the indices are 5 and 17

To simplify the work and limit bandwith usage, the products have been downloaded and stored on the local drive

The products have also already been croped to the correct area with QGIS, as demonstrated in a previous tutorial

The cell below is fully commented, do not execute it. It is here as a reference

Once the products are downloaded we can start our work

# Quick display

We will start by displaying the product on an interactive map to locate the area that we will use in the exercise. To display the product properly, we will need to find the central coordinates of the product as well as those of its boundaries (top left corner & bottom right corner)

Open the True-Colour Image with rasterio

In [ ]:
src_tci_img = rasterio.open(
    "./Products/South_Africa/S2A_MSIL1C_20160901T073612_dry_season/TCI_epsg4326.tif"
)

Now, get the bounds of the tiff files

In [ ]:
x1_tci, y1_tci, x2_tci, y2_tci = src_tci_img.bounds  # Get coordinates of image bounds
print(
    "Bounds of the layer are:\n{0} {1}\n{2} {3}".format(x1_tci, y1_tci, x2_tci, y2_tci)
)

To center the display, you will also need the longitude and latitude of the product : use rasterio to get these values

In [ ]:
lon, lat = src_tci_img.lnglat()  # Get longitude and latitude
m = folium.Map(location=[lat, lon], zoom_start=12)

folium.raster_layers.ImageOverlay(
    image=src_tci_img.read()[0],
    bounds=[[y1_tci, x1_tci], [y2_tci, x2_tci]],
    opacity=0.7,
).add_to(m)

m

The work area is located East of South Africa

# READ DATA

The first step consists in opening all the required data.

The data is stored on the drive, one file for each band, that we will open one by one

## Open files

Using xarray, open the following bands for both WET and DRY regions : 
- B03 (green)
- B04 (red)
- NIR (B08)
- NIR_A (B08A)
- SWIR (B11)

Store the data into two separate Xarrays (one for each season).

In [ ]:
# January 2017 - Wet season
base_path = "./Products/South_Africa/S2A_MSIL1C_20170119T074231_wet_season/"
# blue_wet = xarray.open_rasterio(os.path.join(base_path, "B02.tif"))
green_wet = rioxarray.open_rasterio(
    os.path.join(base_path, "B03_epsg4326.tif")
)  # 554x698
red_wet = rioxarray.open_rasterio(
    os.path.join(base_path, "B04_epsg4326.tif")
)  # 554x698
nir_wet = rioxarray.open_rasterio(
    os.path.join(base_path, "B08_epsg4326.tif")
)  # 554x698
nir_A_wet = rioxarray.open_rasterio(
    os.path.join(base_path, "B08A_epsg4326.tif")
)  # 277x349
swir_wet = rioxarray.open_rasterio(
    os.path.join(base_path, "B11_epsg4326.tif")
)  # 277x349

# Septembre 2016 - Dry saison
base_path_dry = "./Products/South_Africa/S2A_MSIL1C_20160901T073612_dry_season/"
# blue_dry = xarray.open_rasterio(os.path.join(base_path, "B02.tif"))
green_dry = rioxarray.open_rasterio(os.path.join(base_path_dry, "B03_epsg4326.tif"))
red_dry = rioxarray.open_rasterio(os.path.join(base_path_dry, "B04_epsg4326.tif"))
nir_dry = rioxarray.open_rasterio(os.path.join(base_path_dry, "B08_epsg4326.tif"))
nir_A_dry = rioxarray.open_rasterio(
    os.path.join(base_path_dry, "B08A_epsg4326.tif")
)  # 277x349
swir_dry = rioxarray.open_rasterio(
    os.path.join(base_path_dry, "B11_epsg4326.tif")
)  # 277x349

Print the products sizes to make sure they are correctly opened

In [ ]:
print("Red Band dimensions:", red_wet.shape)
print("NIR Band dimensions:", nir_wet.shape)
print("Green Band dimensions:", green_wet.shape)
print("NIR-A Band dimensions:", nir_A_wet.shape)
print("SWIR Band dimensions:", swir_wet.shape)
print("Red Band dimensions:", red_dry.shape)
print("NIR Band dimensions:", nir_dry.shape)
print("Green Band dimensions:", green_dry.shape)
print("NIR-A Band dimensions:", nir_A_dry.shape)
print("SWIR Band dimensions:", swir_dry.shape)

## Create Data Arrays

In this section we will create Data Arrays containing all the products opened above to perform computations

Display one Data Array of your choice to check dimensions, coordinates, band name, etc.

In [ ]:
red_dry

Note that the coordinate **'Band'** has no particular name, so we will need to name each band for all Data Arrays

For each band xarray, rename the band name to an understandable one:
    
e.g. red['band'] = ['red']

In [ ]:
# Affect correct name to bands in the data array
red_dry["band"] = ["red"]
green_dry["band"] = ["green"]
# blue_dry["band"] = ["blue"]
nir_dry["band"] = ["nir"]
nir_A_dry["band"] = ["nir_a"]
swir_dry["band"] = ["swir"]
red_wet["band"] = ["red"]
green_wet["band"] = ["green"]
# blue_wet["band"] = ["blue"]
nir_wet["band"] = ["nir"]
nir_A_wet["band"] = ["nir_a"]
swir_wet["band"] = ["swir"]

Display the same DataArray as above to check its band has been correctly renamed

In [ ]:
red_dry.coords

In the following steps we will compute both NDVI and NDWI.

Note that NDWI uses band B08A and B11 (or B12) while NDVI bands B03 and B08. These bands do not have the same shape : 277x349 vs 554x698, so we can not merge these bands into the same DataArray easily.

This difference in shape is due to the resolution of these bands

Since not all bands yield the same resolution, we will separate the data and therefore concatenate the data into Xarrays : 
- One for Red, NIR, Green bands
- One for NIR-A & SWIR bands

Now, create the 4 data arrays (two per season) by concatenating the required bands

In [ ]:
da_wet_a = xarray.concat([red_wet, green_wet, nir_wet], dim="band")  # For NDVI
da_wet_b = xarray.concat([nir_A_wet, swir_wet], dim="band")  # For NDWI
da_dry_a = xarray.concat([red_dry, green_dry, nir_dry], dim="band")  # For NDVI
da_dry_b = xarray.concat([nir_A_dry, swir_dry], dim="band")  # For NDWI

Clean both data arrays by replacing N/A values by zero

In [ ]:
da_wet_a = da_wet_a.fillna(0)
da_wet_b = da_wet_b.fillna(0)
da_dry_a = da_dry_a.fillna(0)
da_dry_b = da_dry_b.fillna(0)

# Computations

Let's compute NDVI and NDWI

## Compute the NDVI

Remember that:

$$
\text{NDVI} = \frac{Red - NIR}{Red + NIR}
$$

Do not forget to replace N/A values by zero

In [ ]:
NDVI_wet = (da_wet_a.sel(band="nir") - da_wet_a.sel(band="red")) / (
    da_wet_a.sel(band="nir") + da_wet_a.sel(band="red")
)
NDVI_wet = NDVI_wet.fillna(0)  # Clean NDVI data

In [ ]:
NDVI_dry = (da_dry_a.sel(band="nir") - da_dry_a.sel(band="red")) / (
    da_dry_a.sel(band="nir") + da_dry_a.sel(band="red")
)
NDVI_dry = NDVI_dry.fillna(0)

Check correct computation of the NDVI by displaying it with the built-in plot() method

Choose a colormap that will be most adapted

See [matplotlib's documentation](https://matplotlib.org/stable/gallery/color/colormap_reference.html) for more informations

In [ ]:
NDVI_wet.plot(levels=np.arange(0, 0.6, 0.01), cmap="Greens")

In [ ]:
NDVI_dry.plot(levels=np.arange(0, 0.6, 0.01), cmap="Greens")

Notice the variation in vegetation index depending on the season.

During the dry season, we can see that the vegetation has decreased except on the river's border

## Evaluate the average vegetation

Compute the mean value of the NDVI for both seasons, compare these values. What can you conclude ?

In [ ]:
print("The average NDVI for the wet season is: ", NDVI_wet.mean())
print("The average NDVI for the dry season is: ", NDVI_dry.mean())

## Compute the NDWI

Use the following formula 

$$
\text{NDWI} = \frac{(NIR - SWIR)}{NIR + SWIR}
$$



In [ ]:
NDWI_wet = (da_wet_b.sel(band="nir_a") - da_wet_b.sel(band="swir")) / (
    da_wet_b.sel(band="nir_a") + da_wet_b.sel(band="swir")
)
NDWI_wet = NDWI_wet.fillna(0)

Plot the computed NDWI indices for both seasons and compare the results visually

In [ ]:
NDWI_dry = (da_dry_b.sel(band="nir_a") - da_dry_b.sel(band="swir")) / (
    da_dry_b.sel(band="nir_a") + da_dry_b.sel(band="swir")
)
NDWI_dry = NDWI_dry.fillna(0)

In [ ]:
NDWI_wet.plot(levels=np.arange(0, 25, 0.1), cmap="Blues")

In [ ]:
NDWI_dry.plot(levels=np.arange(10, 35, 1), cmap="Blues")

# Interactive plots

## Export the computed NDVI to local files

Get the width and height of the products by opening a reference image (e.g. the red band)

Hint : use **rasterio**

Print the result to make sure the values are correct

In [ ]:
src_img = rasterio.open(
    "./Products/South_Africa/S2A_MSIL1C_20160901T073612_dry_season/B03_epsg4326.tif"
)
width = src_img.width
height = src_img.height
print("Image dimensions:\nwidth {0}px - height {1}px".format(width, height))

Now write the NDVI for wet and dry seasons

In [ ]:
def write_file(src_img, data, outname):
    ndviImage = rasterio.open(
        outname,
        "w",
        driver="Gtiff",
        width=src_img.width,
        height=src_img.height,
        count=1,  # number of bands (1 for NDVI)
        crs=src_img.crs,
        transform=src_img.transform,
        dtype="float64",
    )
    ndviImage.write(data, 1)
    ndviImage.close()  # Do not forget to close opened file !


write_file(src_img, NDVI_wet, "NDVI_wet.tif")
write_file(src_img, NDVI_dry, "NDVI_dry.tif")

Open the written NDVI files to make sure they were properly written

In [ ]:
ndvi_dry_src = rasterio.open("NDVI_dry.tif")
ndvi_wet_src = rasterio.open("NDVI_wet.tif")
show(ndvi_dry_src, cmap="Greens", title="NDVI for DRY SEASON")
show(ndvi_wet_src, cmap="Greens", title="NDVI for WET SEASON")

# Extra work

In this section we will demonstrate how Numba can help speed up computations

To do so, we will compute several vegation indices using different formulae

"Excess Green minus Excess Red" (ExGR) as proposed by Neto et. al.

Though less accurate than NDVI, ExGR (and similar) is useful when the NIR band is not available.

The formula is given below : 

$$
\text{ExGR} = \frac{3G-2.4R-B}{R+G+B}
$$

Where R, G, B correspond to bands Red, Green, Blue respectively

The Enhanced Vegetation Index (EVI2) that is less sensitive than NDVI to biophysical quantities such as vegetation fraction or leaf area index and whose use is therefore weakened with increasing vegetation densities beyond a threshold (see [this article](https://www.tandfonline.com/doi/full/10.1080/17538947.2018.1495770) for example). 

The EVI2 is : 

$$
\text{EVI2} = 2.5 \times \frac{NIR - R}{NIR + (6 - 7.5/2.08)\times R +1}
$$
    
Finally, we will compute the RDVI : 

$$
\text{RDVI} = \frac{NIR - R}{\sqrt{(R + NIR)}}
$$

## Open files

For the sake of this demonstration, we will compute the indices with numpy instead of Xarray

Open all the band files needed to perform the computations using rasterio

In [ ]:
blue_src = rasterio.open(os.path.join(base_path, "B02_epsg4326.tif"))  # 554x698
green_src = rasterio.open(os.path.join(base_path, "B03_epsg4326.tif"))  # 554x698
red_src = rasterio.open(os.path.join(base_path, "B04_epsg4326.tif"))  # 554x698
nir_src = rasterio.open(os.path.join(base_path, "B08_epsg4326.tif"))  # 554x698

blue = blue_src.read()
green = green_src.read()
red = red_src.read()
nir = nir_src.read()

## Computations

First, define a method that will compute the 4 different vegetation indices

In [ ]:
np.seterr(divide="ignore", invalid="ignore")  # suppress warning


def compute_VI_np(blue, green, red, nir):
    """
    This method computes several vegetation indexes
    """
    EVI2 = 2.5 * (nir - red) / ((nir + (6 - 7.5 / 2.08) * red) + 1)
    NDVI = (nir - red) / (nir + red)
    RDVI = (nir - red) / np.sqrt(red + nir)
    ExGR = (3 * green - 2.4 * red - blue) / (red + green + blue)
    return EVI2, NDVI, RDVI, ExGR

Write the same method, but use Numba to perform Just In Time compilation

In [ ]:
from numba import jit


@jit(nopython=True, parallel=True)
def compute_VI_numba(blue, green, red, nir):
    """
    This method computes several vegetation indexes
    """
    EVI2 = 2.5 * (nir - red) / ((nir + (6 - 7.5 / 2.08) * red) + 1)
    NDVI = (nir - red) / (nir + red)
    RDVI = (nir - red) / np.sqrt(red + nir)
    ExGR = (3 * green - 2.4 * red - blue) / (red + green + blue)
    return EVI2, NDVI, RDVI, ExGR

Evaluate the time needed to perform the computations with numpy only

In [ ]:
%%timeit -r 10
EVI2, NDVI, RDVI, ExGR = compute_VI_np(blue, green, red, nir)

In [ ]:
%%timeit -r 10
EVI2, NDVI, RDVI, ExGR = compute_VI_numba(blue, green, red, nir)

Notice that the computation time has been divided by more than 2

## Display results

In [ ]:
EVI2, NDVI, RDVI, ExGR = compute_VI_np(blue, green, red, nir)
from matplotlib import pyplot as plt

formulas = [EVI2, NDVI, RDVI, ExGR]

fig, axs = plt.subplots(nrows=2, ncols=2)
axs[0, 0].imshow(NDVI[0])
axs[0, 0].set_title("NDVI")
axs[0, 1].imshow(RDVI[0])
axs[0, 1].set_title("RDVI")
axs[1, 0].imshow(EVI2[0])
axs[1, 0].set_title("EVI2")
axs[1, 1].imshow(ExGR[0])
axs[1, 1].set_title("ExGR")
plt.tight_layout()